In [ ]:
# for kaggle
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# GPU Configuration

In [ ]:
# Configure amd test GPU
import tensorflow as tf
from tensorflow.python.client import device_lib

# Prevent automatic GPU memory pre-allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

print(tf.__version__)
# print(device_lib.list_local_devices())

In [ ]:
!pip install keras-tuner

In [ ]:
import os

os.chdir('../input/dataset-snr20-outdoor')
print(os.getcwd())

## Encapsulating tuner into a class

## Parameters to be tuned:
- Regularisation and Dropout
    - l2 regularisation on Conv2D layers
    - Dropout layers after Max Pooling layers and before Fully Connected Layer, just before Softmax output
- Learning rate of optimiser
- Batch size
- Number of epochs (Using EarlyStopping keras callback

In [ ]:
from keras.models import Sequential
from keras.utils import np_utils

import h5py
import numpy as np
import math
import os
from tensorflow import keras
from tensorflow.keras import layers

import keras_tuner as kt

In [ ]:
# Get dictionary of RP index and coordinates
# Open HDF5 file and access the dataset
filename = 'dataset_SNR20_outdoor.mat'
hdf5_file = h5py.File(filename, 'r')

features_dataset = hdf5_file['features']
labels_dataset = hdf5_file['labels']['position']

# Convert HDF5 dataset to NumPy array
features = np.array(features_dataset)
labels = np.array(labels_dataset)

# Prepare features for dataset
# Retrieve features from the first UE and transpose the individual matrix
features_transposed = np.zeros((3876,193,16), dtype = np.float64)
for i in range(len(features)):
    features_transposed[i] = features[i][0].T

# Prepare labels for dataset
count = 0
rp_dict = {}
# For labels, have a shape of (1,) where that number represents the class of that coordinate

for label in labels:
    rp_dict[count] = label
    count += 1

# Close the HDF5 file
hdf5_file.close()

In [ ]:
print(os.getcwd())
os.chdir('../augmented-outdoor-dataset')
print(os.getcwd())

In [ ]:
# Load datasets
features = np.load('augmented_features_10_ds.npy')
labels = np.load('augmented_labels_10_ds.npy')

print(f'Shape of features np array: {features.shape}')
print(f'Shape of labels np array: {labels.shape}')

In [ ]:
from sklearn.model_selection import train_test_split

X = features
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# Method 1
class HyperModel(kt.HyperModel):
    
    def build(self,hp):
        # Initialise the weights of neural network layers
        # VarianceScaling is a particular method used to initialise weights
        kaiming_normal = keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='untruncated_normal', seed = 42)

        # Retrieve hyperparameters to be tuned
        l2 = hp.Float("l2", min_value=0, max_value=0.01, step=0.01)
        pooling_dropout = hp.Boolean('conv_dropout', default = False)
        fc_dropout = hp.Boolean('fc_dropout', default = False)
        lr = hp.Float("learning_rate", min_value=0.001, max_value=0.01, step=0.005)
        
        # Make 3x3 convolutional filters
        def conv3x3(x, out_planes, stride=1,name=None):
            x = layers.ZeroPadding2D(padding=1, name=f'{name}_pad')(x)

            # Make 2D convolution layer
            # return layers.Conv2D(filters=out_planes, kernel_size=3, strides=stride, use_bias=False,
            #                     kernel_initializer=kaiming_normal, name=name)(x)
            return layers.Conv2D(filters=out_planes, kernel_size=3, strides=stride, use_bias=False,
                                 kernel_initializer=kaiming_normal, kernel_regularizer=keras.regularizers.L2(l2),
                                 name=name)(x)

        def basic_block(x, planes, stride=1, downsample=None, name=None):
            identity = x

            out = conv3x3(x, planes, stride=stride, name=f'{name}.conv1')
            out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn1')(out)
            out = layers.ReLU(name=f'{name}.relu1')(out)

            out = conv3x3(out, planes, name=f'{name}.conv2')
            out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn2')(out)

            if downsample is not None:
                # Create an identical layer for each layer in the downsample
                for layer in downsample:
                    identity = layer(identity)

            # Performs element-wise addition of multiple inputs. It is used to combine or merge the outputs of two or more layers by adding them together
            out = layers.Add(name=f'{name}.add')([identity, out])    
            out = layers.ReLU(name=f'{name}.relu2')(out)

            return out

        def make_layer(x, planes, blocks, stride=1, name=None):
            downsample = None

            # inplanes refer to the number of channels in filters
            inplanes = x.shape[3]

            # Check whether we are downsampling our data (i.e. not going through every elememt)
            # This happens under two circumstances:
            # 1. when stride != 1
            # 2. when the layer we want to make has no. of channels less than the no. of channels input has
            if stride != 1 or inplanes != planes:
                # downsample consists of a Conv2D layer and a BatchNormalization layer
                downsample = [
                    layers.Conv2D(filters=planes, kernel_size=1, strides=stride, use_bias=False, kernel_initializer=kaiming_normal,
                                  kernel_regularizer=keras.regularizers.L2(l2),
                                  name=f'{name}.0.downsample.0'),
#                    layers.Conv2D(filters=planes, kernel_size=1, strides=stride, use_bias=False, kernel_initializer=kaiming_normal,
#                                  name=f'{name}.0.downsample.0'),
                    layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.0.downsample.1'),
                ]

            # If no downsample, downsample = None

            x = basic_block(x, planes, stride, downsample, name=f'{name}.0')
            for i in range(1, blocks):
                x = basic_block(x, planes, name=f'{name}.{i}')

            return x

        def resnet(x, blocks_per_layer, num_classes=1000):

            # ---------------------------------
            # Initial entry block
            x = layers.ZeroPadding2D(padding=3, name='conv1_pad')(x)
            x = layers.Conv2D(filters=64, kernel_size=7, strides=2, use_bias=False,
                              kernel_initializer=kaiming_normal,
                              kernel_regularizer=keras.regularizers.L2(l2),
                              name='conv1')(x)
            # x = layers.Conv2D(filters=64, kernel_size=7, strides=2, use_bias=False,
            #                  kernel_initializer=kaiming_normal,
            #                  name='conv1')(x)
            x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='bn1')(x)
            x = layers.ReLU(name='relu1')(x)
            x = layers.ZeroPadding2D(padding=1, name='maxpool_pad')(x)
            x = layers.MaxPool2D(pool_size=3, strides=2, name='maxpool')(x)

            # Additional - Addition of a dropout layer
            if pooling_dropout:
                x = layers.Dropout(rate = 0.2)(x)   
            # ---------------------------------

            # ---------------------------------
            # This block of code creates the ResNet blocks
            # In ResNet-18, only have 2 layers per block
            x = make_layer(x, 64, blocks_per_layer[0], name='layer1')
            x = make_layer(x, 128, blocks_per_layer[1], stride=2, name='layer2')
            x = make_layer(x, 256, blocks_per_layer[2], stride=2, name='layer3')
            x = make_layer(x, 512, blocks_per_layer[3], stride=2, name='layer4')
            # ---------------------------------

            # ---------------------------------
            x = layers.GlobalAveragePooling2D(name='avgpool')(x)

            # Addition of a dropout layer
            if pooling_dropout:
                x = layers.Dropout(rate = 0.2)(x)    

            initializer = keras.initializers.RandomUniform(-1.0 / math.sqrt(512), 1.0 / math.sqrt(512))
            x = layers.Dense(units=num_classes, kernel_initializer=initializer, bias_initializer=initializer, name='fc')(x)

            # Additional - Addition of a dropout layer
            if fc_dropout:
                x = layers.Dropout(rate = 0.5)(x)  

            # Softmax output layer
            x = layers.Dense(units=num_classes, activation='softmax')(x)  
            # ---------------------------------

            return x

        def resnet18(x, **kwargs):
            return resnet(x, [2, 2, 2, 2], **kwargs)

        def resnet34(x, **kwargs):
            return resnet(x, [3, 4, 6, 3], **kwargs)

        # Create model
        model_inputs = keras.Input(shape = (193, 16, 1))
        model_outputs = resnet18(model_inputs, num_classes = 3876)
        resnet18_model = keras.Model(model_inputs, model_outputs)

        # Compile model - Classification
        # default learning rate is 1e-3 = 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        
        resnet18_model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

        return resnet18_model
    
    def fit(self, hp, model, X_train, y_train, validation_data = None, **kwargs):
        
        return model.fit(X_train, y_train,
                        validation_data = validation_data,
                        batch_size = hp.Choice('batch_size', [16,32,64]),
                        **kwargs,
                        )

In [ ]:
# Must change back to working directory for below code to run
print(os.getcwd())
os.chdir('../../working')
print(os.getcwd())

In [ ]:
# Method 1
tuner = kt.RandomSearch(
    HyperModel(),
    objective = 'val_loss',
    max_trials = 30,
)

In [ ]:
tuner.search_space_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train, y_train,
            validation_data = (X_test, y_test),
            epochs = 100,
            callbacks = [stop_early])

In [ ]:
tuner.results_summary()

## Analysis of results

In [ ]:
# Analysis of results
import os
import json

print(os.getcwd())

for _, _, file_names in os.walk('resnet18_tuningresults'):
    files = file_names    

tuning_results = {}

for file in files:
    cur_filename = 'resnet18_tuningresults/' + file
    data = open(cur_filename)
    data = json.load(data)
    
    trial_results = {}
    trial_results['trial_id'] = data['trial_id']
    trial_results['values'] = data['hyperparameters']['values']
    trial_results['val_loss'] = data['metrics']['metrics']['val_loss']['observations'][0]['value'][0]
    trial_results['status'] = data['status']
    tuning_results[data['trial_id']] = trial_results

In [ ]:
best_trial = min(tuning_results.keys(), key=lambda x: tuning_results[x]['val_loss'])

print(f'Trial that resulted in minimum validation loss: {best_trial}')
print(f'Validation Loss: {tuning_results[best_trial]["val_loss"]}')
print(f'Best Hyperparameters: {tuning_results[best_trial]["values"]}')

## Best Hyperparameters:
- l2: 0.0
- conv_dropout: False
- fc_dropout: False
- learning_rate: 0.001
- batch_size: 64